In [1]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def _my_hack_init(self, text):
        self.text = text
        self.prepared_text = ''
        self.html_tags_offsets = []
        self.raw_text = ''
        self.prev_text_end = 0
    
    def handle_data(self, data):
        
        text_start = self.getpos()[1]
        text_end = text_start + len(data)
        
        # TODO: there are some encoding problem, try to parse this:
        # https://wiki.swap.online/decentralized-exchange-of-bitcoins-for-altcoins-right-in-the-browser-live-demo
        # if self.text[text_start: text_end] != data:
        #     print(self.text[text_start: text_end], '|||', data)
        #     print((len(self.raw_text), self.text[self.prev_text_end: text_start]))
        #     print()
        
        self.html_tags_offsets.append((len(self.raw_text), self.text[self.prev_text_end: text_start]))
        self.prev_text_end = text_end
        self.raw_text += self.text[text_start: text_end]  # can't use 'data' - encoding problem

def extract_tags(html_string):
    # removing '\n', '\t', '  '
    html_string = ' '.join(html_string.split())

    parser = MyHTMLParser()
    parser._my_hack_init(html_string)
    parser.feed(html_string)
    return (parser.html_tags_offsets, parser.raw_text)

In [122]:
from newspaper import Article
def parse(url):
    try:
        a = Article(url, keep_article_html=True)
        a.download()
        a.parse()
        a.nlp()

        return  {
            "author": ", ".join(a.authors),
            "source": a.source_url[a.source_url.find("//") + 2:].split("/")[0],
            "title": a.title,
            "image": a.top_image,
            "url": a.url,
            "publishedAt": a.publish_date,
            "html": a.article_html,
            "text": a.text,
            "summary": a.summary,
            "keywords": a.keywords,
        }
    except Exception as e:
        print(e)
        return { 'error': True, 'description': "'%s' parsing went wrong with error: '%s'" % (url, e)}

In [123]:
%%time
url = 'https://itnext.io/anyway-heres-how-to-do-ajax-api-calls-with-vue-js-e71e57d5cf12'
article = parse(url)
html = article['html']

CPU times: user 271 ms, sys: 43.7 ms, total: 314 ms
Wall time: 1.91 s


_____
_____

In [130]:
_html = html[3000:6000]
print(_html)
print(len(_html))
html_string = _html

<br>anti-pattern</li></ul><p class="aspectRatioPlaceholder-fill"></p><img class="graf-image" src="https://cdn-images-1.medium.com/max/1600/1*7Hkh8kNcJqIOAm3B59qsRg.png">From Vuex&#8217;s documentation<ul class="postList"><li name="4ec6" id="4ec6" class="graf graf--li graf-after--figure"><strong class="markup--strong markup--li-strong">Components</strong>: You could directly make AJAX calls in the methods of your component<br>- <strong class="markup--strong markup--li-strong">PROS</strong>: You just use what you need and have no overload<br>- <strong class="markup--strong markup--li-strong">CONS</strong>: Your calls are scattered everywhere and it can become really hard to maintain. You can create code duplication if two components need the same API call</li></ul><p name="1adf" id="1adf" class="graf graf--p graf-after--li">As you can see, <strong class="markup--strong markup--p-strong">all of them have pros and cons</strong> so which one is the best?<br>Well, none of them, or should I s

In [131]:
def unescape_xml_tags(text):
    # TODO: implement proper XML tags replace 
    # html_string = re.sub("&#?\w+;", "'", html_string)
    text = re.sub('&#8217;', "'", text)
    text = re.sub('&#160;', " ", text)
    text = re.sub('&#8230;', "…", text)
    return text

html_string = unescape_xml_tags(html_string)

In [132]:
html_string

'<br>anti-pattern</li></ul><p class="aspectRatioPlaceholder-fill"></p><img class="graf-image" src="https://cdn-images-1.medium.com/max/1600/1*7Hkh8kNcJqIOAm3B59qsRg.png">From Vuex\'s documentation<ul class="postList"><li name="4ec6" id="4ec6" class="graf graf--li graf-after--figure"><strong class="markup--strong markup--li-strong">Components</strong>: You could directly make AJAX calls in the methods of your component<br>- <strong class="markup--strong markup--li-strong">PROS</strong>: You just use what you need and have no overload<br>- <strong class="markup--strong markup--li-strong">CONS</strong>: Your calls are scattered everywhere and it can become really hard to maintain. You can create code duplication if two components need the same API call</li></ul><p name="1adf" id="1adf" class="graf graf--p graf-after--li">As you can see, <strong class="markup--strong markup--p-strong">all of them have pros and cons</strong> so which one is the best?<br>Well, none of them, or should I say: 

In [112]:
def unescape(text):
    return re.sub("&#?\w+;", text)

unescape(html_string)

TypeError: sub() missing 1 required positional argument: 'string'

In [107]:
import html

s = html.unescape(html_string)
s

'</p><p name="2ea6" id="2ea6" class="graf graf--p graf-after--p">For each API resource, create an API file with an object literal or a static class or just export functions that make the API calls.</p><h4 name="eefa" id="eefa" class="graf graf--h4 graf-after--p">Why is this\xa0better?</h4><ul class="postList"><li name="5629" id="5629" class="graf graf--li graf-after--h4">If you need it in another project that isn’t a Vue project, well: it’s reusable\xa0! It’s just a JS file, right?</li><li name="348d" id="348d" class="graf graf--li graf-after--li">If a path / url changes in the API, it’s centralised so no need to remember in which action it was called or in which component, just go to your API file and make the changes there\xa0:)</li><li name="4ebd" id="4ebd" class="graf graf--li graf-after--li">Abstraction is always better than tightly coupled logic</li></ul><p name="fd5b" id="fd5b" class="graf graf--p graf-after--li">Here are some screens to understand what I’m saying.</p><p class="

In [41]:
import re

html_string = _html

def extract_tags(html_string):
    raw_text = ""
    length_of_cutted_tokens = 0
    end_of_prev_token = 0
    html_tags_offsets = []
    for match in re.finditer('<.*?>', html_string):
        s, e, g = match.start(), match.end(), match.group()

        token = html_string[s: e]
        raw_text += html_string[end_of_prev_token: s] + ' '
        html_tags_offsets.append((len(raw_text), g))

        end_of_prev_token = match.end()
        length_of_cutted_tokens += len(token)

    html_tags_offsets = merge_tags(html_tags_offsets)

    return (html_tags_offsets, raw_text)

'  For each API resource, create an API file with an object literal or a static class or just export functions that make the API calls.  Why is this&#160;better?   If you need it in another project that isn&#8217;t a Vue project, well: it&#8217;s reusable&#160;! It&#8217;s just a JS file, right?  If a path / url changes in the API, it&#8217;s centralised so no need to remember in which action it was called or in which component, just go to your API file and make the changes there&#160;:)  Abstraction is always better than tightly coupled logic   Here are some screens to understand what I&#8217;m saying.    This is a fresh&#160;install   This is the main.js file where I config axios, the AJAX library (it could be any other library or fetch&#160;API) '

In [42]:
def merge_tags(html_tags_offsets):
    merged = []
    done_offsets = []
    for t1 in html_tags_offsets:
        if t1[0] in done_offsets:
            continue
        tags_with_same_offset = [t for o, t in html_tags_offsets if o == t1[0]]
        new_tag = "".join(tags_with_same_offset)
        merged.append((t1[0], new_tag))  
        done_offsets.append(t1[0])
    return merged

def insert_tags(html_tags_offsets, raw_text):
    text_with_html = raw_text
    shift = 0
    for offset, tag_text in sorted(html_tags_offsets, key=lambda x: -x[0]):
        shifted_offset = offset 
        # -1 -> remove space that was added above
        text_with_html = text_with_html[:shifted_offset - 1] + tag_text + text_with_html[shifted_offset:]
    return text_with_html

new_html = insert_tags(html_tags_offsets, raw_text)

In [43]:
print(len(new_html))
new_html

1458


'</p><p name="2ea6" id="2ea6" class="graf graf--p graf-after--p">For each API resource, create an API file with an object literal or a static class or just export functions that make the API calls.</p><h4 name="eefa" id="eefa" class="graf graf--h4 graf-after--p">Why is this&#160;better?</h4><ul class="postList"><li name="5629" id="5629" class="graf graf--li graf-after--h4">If you need it in another project that isn&#8217;t a Vue project, well: it&#8217;s reusable&#160;! It&#8217;s just a JS file, right?</li><li name="348d" id="348d" class="graf graf--li graf-after--li">If a path / url changes in the API, it&#8217;s centralised so no need to remember in which action it was called or in which component, just go to your API file and make the changes there&#160;:)</li><li name="4ebd" id="4ebd" class="graf graf--li graf-after--li">Abstraction is always better than tightly coupled logic</li></ul><p name="fd5b" id="fd5b" class="graf graf--p graf-after--li">Here are some screens to understand 

In [44]:
print(len(_html))
_html

1458


'</p><p name="2ea6" id="2ea6" class="graf graf--p graf-after--p">For each API resource, create an API file with an object literal or a static class or just export functions that make the API calls.</p><h4 name="eefa" id="eefa" class="graf graf--h4 graf-after--p">Why is this&#160;better?</h4><ul class="postList"><li name="5629" id="5629" class="graf graf--li graf-after--h4">If you need it in another project that isn&#8217;t a Vue project, well: it&#8217;s reusable&#160;! It&#8217;s just a JS file, right?</li><li name="348d" id="348d" class="graf graf--li graf-after--li">If a path / url changes in the API, it&#8217;s centralised so no need to remember in which action it was called or in which component, just go to your API file and make the changes there&#160;:)</li><li name="4ebd" id="4ebd" class="graf graf--li graf-after--li">Abstraction is always better than tightly coupled logic</li></ul><p name="fd5b" id="fd5b" class="graf graf--p graf-after--li">Here are some screens to understand 

In [39]:
print(_html == new_html)

True


In [80]:
url = "https://ai.morejust.media/parse?url=https://medium.com/swaponline/atomic-swap-with-usdt-swap-online-solution-in-two-hundred-lines-of-code-c2ddb7b8886a?source=collection_home---4------9---------------------"

In [81]:
import requests

In [82]:
r = requests.get(url)
j = r.json()

In [83]:
summary = j["summary"]
text = j["text"]

In [84]:
summary

'Atomic Swap with USDT: Swap Online solution in two hundred lines of codeOn the eve of the release on the mainnet, the team of the cross-chain wallet Swap Online is publishing a research study and the code of the atomic swap using USDT.\nTether (symbol Tether — ₮, ticker — USDT) is a hybrid cryptocurrency with a rate binding to one US dollar.\nThis fall, the team of Swap Online is ready to present an atomic swap with Tether.\nThe payload is a mandatory part of any Omni transaction, as it is a sequence of bytes containing all the necessary information about the transaction.\nLet us consider what information is stored in the payload itselftransaction marker — 4 bytes, the mandatory part of any Omni payload is always equal to 0x6f6d6e69 — ASCII code omni.'

In [85]:
text

'Atomic Swap with USDT: Swap Online solution in two hundred lines of code\n\nOn the eve of the release on the mainnet, the team of the cross-chain wallet Swap Online is publishing a research study and the code of the atomic swap using USDT.\n\nUSD Tether — the equivalent of the dollar on Omni Layer\n\nThe solution described above with the protocol “over” the Bitcoin network gave life to one of the most controversial cryptocurrency projects of the last two years — Tether. Tether (symbol Tether — ₮, ticker — USDT) is a hybrid cryptocurrency with a rate binding to one US dollar. Moreover, according to the assurances of Tether Limited, the issuer of the given tokens, the “binding” is to be understood literally, as each purchased token of USDT corresponds to one US dollar available at the disposal of the company.\n\nIf we take the three largest exchanges based on their daily turnover of transactions at the time of writing (Binance, OKEx and HuObi), and then track the five most popular tradi

In [86]:
from difflib import SequenceMatcher 


def find_longest_substring(s1, s2): 
    # returns: offset according s1, substring itself
    seqMatch = SequenceMatcher(None, s1, s2) 
    match = seqMatch.find_longest_match(0, len(s1), 0, len(s2)) 
    
    if (match.size != 0): 
        return (match.a, s1[match.a: match.a + match.size])
    return None, None

In [89]:
"asdasd".replace("asd", '')

''

In [94]:
import csv
fakes = []
with open('./data/fakes.csv', 'r') as f:
    for row in csv.reader(f):
        fakes.append(str(row[0]))
print(fakes)

['associatedmediacoverage.com', 'baltimoregazette.com', 'beforeitsnews.com', 'bients.com', 'bizstandardnews.com', 'civictribune.com', 'civictribune.com', 'creambmp.com', 'corruptmodi.com', 'CountyNewsroom.info', 'dailybuzzlive.comdcgazette.com', 'drudgereport.com.co', 'empirenews.net', 'enduringvision.com', 'indecisionforever.com', 'mediamass.net', 'megynkelly.us', 'msnbc.com.co', 'msnbc.website', 'nationalreport.net', 'news-hound.com', 'newslo.com', 'newsmutiny.com', 'politicalo.com', 'realnewsrightnow.com', 'rilenews.com', 'usatoday.com.co', 'thebostontribune.com']


In [96]:
with open('./data/fakes.csv', 'r') as f:
    fakes = f.readlines()
    fakes = [f.strip() for f in fakes]
print(fakes)

['associatedmediacoverage.com', 'baltimoregazette.com', 'beforeitsnews.com', 'bients.com', 'bizstandardnews.com', 'civictribune.com', 'civictribune.com', 'creambmp.com', 'corruptmodi.com', 'CountyNewsroom.info', 'dailybuzzlive.comdcgazette.com', 'drudgereport.com.co', 'empirenews.net', 'enduringvision.com', 'indecisionforever.com', 'mediamass.net', 'megynkelly.us', 'msnbc.com.co', 'msnbc.website', 'nationalreport.net', 'news-hound.com', 'newslo.com', 'newsmutiny.com', 'politicalo.com', 'realnewsrightnow.com', 'rilenews.com', 'usatoday.com.co', 'thebostontribune.com']


In [91]:
def extract_facts(post):
    text = post["text"]
    summary = post["summary"]
    result = []
    while True:
        offset, substring = find_longest_substring(text, summary)
        if offset is None:
            break
        if len(substring) < 20:
            break
            
        result.append({ "offset": offset, "phrase": substring})

        summary = summary.replace(substring, "")    
        if len(summary) < 50:
            break

        print({"offset": offset, "phrase": substring})
    return result

extract_facts({"text": text, "summary": summary})

{'offset': 74, 'phrase': 'On the eve of the release on the mainnet, the team of the cross-chain wallet Swap Online is publishing a research study and the code of the atomic swap using USDT.\n'}
{'offset': 2625, 'phrase': 'transaction marker — 4 bytes, the mandatory part of any Omni payload is always equal to 0x6f6d6e69 — ASCII code omni.'}


[{'offset': 74,
  'phrase': 'On the eve of the release on the mainnet, the team of the cross-chain wallet Swap Online is publishing a research study and the code of the atomic swap using USDT.\n'},
 {'offset': 2625,
  'phrase': 'transaction marker — 4 bytes, the mandatory part of any Omni payload is always equal to 0x6f6d6e69 — ASCII code omni.'}]